# 第三章 LangChain 表达式语言 LangChain Expression Language

在这一章我们会介绍 LangChain Expression Language（或称为 LCEL），被称之为 Langchain 的表达式语言。LCEL 是一种新的语法，是 LangChain 工具包的重要补充，他有许多优点，使得我们处理 LangChain 和代理更加简单方便。

1. LCEL 提供了异步、批处理和流处理支持，这使得代码多功能化，并且代码可以快速在不同服务器中应用和运行。
    - 异步：程序可以同时执行多个任务，而不是按照顺序一个接一个地执行
    - 批处理：是一种将一组任务或数据作为一个批次进行处理的方法，而不是逐个处理
    - 流式处理：理是一种连续处理数据的方法，数据会持续不断地进入系统并被处理，流式处理能够在数据到达时立即进行处理，并且可以以持续且低延迟的方式处理数据。

2. LCEL 拥有 fallbacks 措施，也叫回退安全机制，有时LLM得到的结果不可控，这时你可以将结果进行回退，甚至可以附加到整个链上

3. LCEL 增加了 LLM 的并行性，LLM 运行通常是耗费时间的，并行可以加快得到结果的速度。

4. LCEL 内置了日志记录，记录代理的运行情况。即使代理复杂，日志也有助于理解复杂链条和代理的运行情况。

在前面的课程中，我们知道 LangChain 提供了组件 链（chain） 可以将组件组合起来发挥 LLM 更强大的功能，但是语法非常复杂。在这里，LCEL 提供了一种管道语法，使从基本组件构建复杂链变得容易，我们可以通过 LangChain 完成`Chain = prompt | LLM |OutputParser `的组合，具体使用我们将在下文内容中讨论。链（Chains）通常将大语言模型（LLM）与提示（Prompt）结合在一起，基于此，我们可以对文本或数据进行一系列操作。

![image.png](attachment:image.png)

- [一、简单链 Simple Chain](#一、简单链-Simple-Chain)
- [二、更复杂的链 More complex chain](#二、更复杂的链-More-complex-chain)
  - [1.1 构建简单向量数据库](#1.1-构建简单向量数据库)
  - [1.2 使用RunnableMap](#1.2-使用RunnableMap)
- [三、绑定 Bind](#三、绑定-Bind)
  - [3.1 单函数绑定](#3.1-单函数绑定)
  - [3.2 多个函数绑定](#3.2-多个函数绑定)
- [四、后备措施 Fallbacks](#四、后备措施-Fallbacks)
  - [4.1 使用早期模型格式化输出](#4.1-使用早期模型格式化输出)
  - [4.2 使用新模型格式化输出](#4.2-使用新模型格式化输出)
  - [4.3 fallbacks方法](#4.3-fallbacks方法)
- [五、接口 Interface](#五、接口-Interface)
  - [5.1 invoke接口](#5.1-invoke接口)
  - [5.2 batch接口](#5.2-batch接口)
  - [5.3 stream接口](#5.3-stream接口)
  - [5.4 异步接口](#5.4-异步接口)
- [六、英文版提示](#六、英文版提示)

## 一、简单链 Simple Chain

接下来我们依旧会使用 OpenAI 的 API，所以首先我们要初始化我们的 API_Key，这个方法和上一章的方式是一样的。

In [28]:
# !pip install langchain
# !pip install langchain_openai
# !pip install langchain-community
# !pip install openai==0.28
# !pip install "langchain[docarray]"
# !pip install docarray
# !pip install tiktoken

In [1]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "your_api_key"
openai.api_key = os.environ['OPENAI_API_KEY']

接下来首先导入 LangChain 的库，并且定义一个简单的链，这个链包括提示模板，大语言模型和一个输出解析器。我们可以看到，成功输出了大语言模型的结果，完成了一个简单的链。

In [2]:
# 导入LangChain所需的模块
from langchain.prompts import ChatPromptTemplate
# from langchain.chat_models import ChatOpenAI
# 最新的openai版本中，要使用“from langchain_openai import ChatOpenAI”替代“from langchain.chat_models import ChatOpenAI”
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# 使用 ChatPromptTemplate 从模板创建一个提示，模板中的 {topic} 将在后续代码中替换为实际的话题
prompt = ChatPromptTemplate.from_template(
    "告诉我一个关于{topic}的短笑话"
)

# 创建一个 ChatOpenAI 模型实例，默认使用 gpt-3.5-turbo 模型
model = ChatOpenAI()

# 创建一个StrOutputParser实例，用于解析输出
output_parser = StrOutputParser()

# 创建一个链式调用，将 prompt、model 和output_parser 连接在一起
chain = prompt | model | output_parser

# 调用链式调用，并传入参数
chain.invoke({"topic": "熊"})

'为什么熊不喜欢雨天？因为它们怕变成“湿熊”！哈哈哈！'

如果我们去查看`Chain`的输出，我们会发现，他跟我们定义的是一样的，一共有三部分进行组成，也就是`Chain = prompt | LLM |OutputParser `。`|`符号类似于 unix 管道操作符，它将不同的组件链接在一起，将一个组件的输出作为输入提供给下一个组件。在这个链中，用户输入被传递给提示模板，然后提示模板输出被传递给模型，然后模型输出被传递到输出解析器。

In [3]:
# 查看Chain的值
chain

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='告诉我一个关于{topic}的短笑话'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x131c27650>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x131c5cec0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

## 二、更复杂的链 More complex chain

接下来，我们会创建一个更复杂的链条，在之前的课程中，我们接触过如何进行检索增强生成。所以接下来我们使用 LCEL 来重复之前的过程，将用户的问题和向量数据库检索结果结合起来，使用 RunnableMap 来构建一个更复杂的链。

### 2.1 构建简单向量数据库
首先我们构建一个向量数据库，这个简单的向量数据库只包含两句话，使用 OpenAI 的 Embedding 作为嵌入模型，然后我们通过 `vector store.as_retriever `来创建一个检索器。

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

# 创建一个DocArrayInMemorySearch对象，用于存储和搜索文档向量
vectorstore = DocArrayInMemorySearch.from_texts(
    ["哈里森在肯肖工作", "熊喜欢吃蜂蜜"],
    embedding=OpenAIEmbeddings() # 使用OpenAI的Embedding
)

# 创建一个检索器
retriever = vectorstore.as_retriever()

/Users/jingllinzhou/anaconda3/envs/dw/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


通过之前的学习，如果我们调用`retriever.get_relevant_documents`，我们会得到相关的检索文档，首先我们问“哈里森在哪里工作？”，我们会发现返回了一个文档列表，他会根据相似度排序返回文档列表，所以其中最相关的放在了第一个。

In [5]:
# 获取与问题“哈里森在哪里工作？”相关的文档
retriever.get_relevant_documents("哈里森在哪里工作？")

/Users/jingllinzhou/anaconda3/envs/dw/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='哈里森在肯肖工作'), Document(page_content='熊喜欢吃蜂蜜')]

如果我们换一个问题，比如"熊喜欢吃什么"，可以看到问题的顺序就发生了变化。

In [6]:
# 获取与问题“熊喜欢吃什么”相关的文档
retriever.get_relevant_documents("熊喜欢吃什么")

[Document(page_content='熊喜欢吃蜂蜜'), Document(page_content='哈里森在肯肖工作')]

### 3.2 使用RunnableMap

上述例子返回两个结果是因为只有两个文档列表，这完全适用于更多文档情况。接下来我们会加入`RunnableMap`，在这个`RunnableMap`中，不仅仅有用户的问题，以及有对应的问题的文档列表，相当于这也为大模型的文档增加了上下文，这样就能完成检索增强的事情。如果我们正常问一个问题，可以看到，大模型正确的返回了文档里面的结果，得到了正确的输出。

In [7]:
from langchain.schema.runnable import RunnableMap

# 定义一个模板字符串template
template = """仅根据以下上下文回答问题：
{context}

问题：{question}
"""

# 使用 template 作为模板
prompt = ChatPromptTemplate.from_template(template)

# 创建一个处理链 chain ，包含了 RunnableMap、prompt、model 和 output_parser 组件
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

# 调用chain的invoke方法
chain.invoke({"question": "哈里森在哪里工作?"})

'哈里森在肯肖工作。'

如果我们想更深入挖掘一下背后的工作机理，我们可以看一下`RunnableMap`，我们把其创建为一个输入，用一样的方式进行操作。我们可以看到，在这之中，`RunnableMap`提供了`context`和`question`两个变量，一个是查询的文档列表，另一个是对应的问题，这个大模型就可以根据提供文档来总结回答对应的问题了。

In [8]:
# 创建一个RunnableMap对象，其中包含两个键值对
# 键 "context" 对应一个lambda函数，用于获取相关文档，函数输入参数为x，即输入的字典，函数返回值为retriever.get_relevant_documents(x["question"])
# 键 "question" 对应一个lambda函数，用于获取问题，函数输入参数为x，即输入的字典，函数返回值为x["question"]
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

# 调用 inputs 的 invoke 方法，并传递一个字典作为参数，字典中包含一个键值对，键为"question"，值为"哈里森在哪里工作?"
inputs.invoke({"question": "哈里森在哪里工作?"})

{'context': [Document(page_content='哈里森在肯肖工作'),
  Document(page_content='熊喜欢吃蜂蜜')],
 'question': '哈里森在哪里工作?'}

## 三、绑定 Bind

在上一章我们介绍了OpenAI函数的调用，新的`function`参数可以自动判断是否要使用工具函数，如果需要就会返回需要使用的参数。接下来我们也使用LangChain实现OpenAI函数调用的新功能，首先需要一个函数的描述信息，以及定义函数，这里的函数还是使用上一章的`get_current_weather`函数。

In [9]:
# 定义一个函数
functions = [
  {
    "name": "get_current_weather",
    "description": "获取指定位置的当前天气情况",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "城市和省份，例如：北京，北京市",
        },
        "unit": {"type": "string", "enum": ["摄氏度", "华氏度"]},
      },
      "required": ["location"],
    },
  }
]

### 3.1 单函数绑定

接下来我们使用`bind`的方法把工具函数绑定到大模型上，并构建一个简单的链。进行调用以后，我们可以看到返回了一个`AIMessage`，	其中返回的`content`为空，但是返回了我们需要调用工具函数的参数。

In [10]:
# 使用ChatPromptTemplate.from_messages方法创建一个ChatPromptTemplate对象
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

# 使用bind方法绑定functions参数
model = ChatOpenAI(temperature=0).bind(functions=functions)

runnable = prompt | model

# 调用invoke方法
runnable.invoke({"input": "北京天气怎么样？"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"location":"北京","unit":"摄氏度"}', 'name': 'get_current_weather'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 94, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a6c821a8-f656-4d9e-8698-7b1a29d8f3f6-0', usage_metadata={'input_tokens': 94, 'output_tokens': 23, 'total_tokens': 117})

### 3.2 多个函数绑定

同时我们也可以定义多个`function`，大模型在对话的时候可以自动判断使用哪一个函数。这里面我们定义有两个函数，第一个函数是类似于前面的`weather_search`，搜索给定机场的天气，然后我们还定义了一个赛事体育新闻搜索的`sports_search`，查询天气的函数`weather_search`接受的参数为airport_code即机场代码，体育新闻搜索函数`sports_search`接受的参数为team_name即体育队名。由于这里我们不需要运行这些函数，因为大模型是通过问的问题来自动判断是否调用这些函数，并且返回参数，并不会直接帮我们调用。

In [11]:
functions = [
    {
        "name": "weather_search",
        "description": "搜索给定机场代码的天气",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "要获取天气的机场代码"
                },
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "搜索最近体育赛事的新闻",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "要搜索的体育队名"
                },
            },
            "required": ["team_name"]
        }
    }
]

接着我们就可以使用函数绑定大模型，定义一个简单的链，我们可以看到，当我们问了相关的问题以后，大模型能够自动判断并且正确返回参数，知道需要调用函数了。

In [12]:
# 绑定大模型
model = model.bind(functions=functions)
runnable = prompt | model

runnable.invoke({"input": "爱国者队昨天表现的怎么样?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"爱国者队"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 125, 'total_tokens': 145}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2bc5c085-bba4-4feb-acb5-0396eba7e362-0', usage_metadata={'input_tokens': 125, 'output_tokens': 20, 'total_tokens': 145})

## 四、后备措施 Fallbacks

在使用早期的OpenAI模型如"text-davinci-001"，这些模型在对话过程中，不支持格式化输出结果即它们都是以字符串的形式输出结果，这对我们有时候需要解析 LLM 的输出带来一些麻烦，比如下面这个例子，就是利用早期模型"text-davinci-001"来回答用户的问题，我们希望 llm 能以 json 格式输出结果。

我们定义了 OpenAI 的模型以及创建了一个简单的链，以此加入 json 希望能以 json 格式输出结果，我们让 simple_model 写三首诗，并以 josn 格式输出，每首诗必须包含:`标题，作者和诗的第一句`。我们会发现结果只有字符串，无法输出指定格式的内容，虽然里面有一些`[`，但是本质上还是一个大的字符串，这就无法让我们解析输出。

> 由于OpenAI于2024年1月4日停用了模型text-davinci-001，你将使用OpenAI推荐的替代模型gpt-3.5-turbo-instruct。

在使用语言模型时，你可能经常会遇到来自底层 API 的问题，无论这些问题是速率限制还是停机时间。因此，当你将 LLM 应用程序转移到实际生产环境中时，防范这些问题变得越来越重要。这就是为什么我们引入了`回退（Fallbacks）`的概念。

### 4.1 使用早期模型格式化输出

In [18]:
from langchain.llms import OpenAI
import json

# 使用早期的OpenAI模型
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

challenge = "写三首诗，并以josn格式输出，每首诗必须包含:标题，作者和诗的第一句。"

simple_model.invoke(challenge)

'\n\n{\n  "title": "春风",\n  "author": "李白",\n  "first_line": "春风又绿江南岸",\n  "content": [\n    "春风又绿江南岸",\n    "花开满树柳如丝",\n    "鸟儿欢唱天地宽",\n    "人间春色最宜人"\n  ]\n}\n\n{\n  "title": "夜雨",\n  "author": "杜甫",\n  "first_line": "夜雨潇潇",\n  "content": [\n    "夜雨潇潇",\n    "孤灯照旧",\n    "思念如潮",\n    "泛滥心头"\n  ]\n}\n\n{\n  "title": "山行",\n  "author": "王维",\n  "first_line": "远上寒山石径斜",\n  "content": [\n    "远上寒山石径斜",\n    "白云生处有人家",\n    "停车坐爱枫林晚",\n    "霜叶红于二月花"\n  ]\n}'

如果我们使用`simple_chain`来运行，我们就会发现出现了 json 解码错误的问题，因为返回的结果就是一个字符串，无法解析，所以下面代码就会报错。

In [19]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 15 column 1 (char 147)

### 4.2 使用新模型格式化输出

所以我们会发现早期版本的 OpenAI 模型不支持格式化的输出，所以即使使用 LangChain 并且加上了`json.load`但是还是会出现错误，但是如果我们使用新的`gpt-3.5-turbo`模型就不会出现这个问题。

In [25]:
# 默认使用新的模型
model = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
# 注意这里要使用支持聊天功能的模型，如：gpt-3.5-turbo 或 gpt-4
chain = model | StrOutputParser() | json.loads

chain.invoke(challenge)

{'poem1': {'title': '晨曦',
  'author': '张三',
  'first_line': '清晨的第一缕阳光',
  'content': '清晨的第一缕阳光，洒在窗前的花朵上，唤醒了沉睡的大地。'},
 'poem2': {'title': '夜色',
  'author': '李四',
  'first_line': '夜色笼罩大地',
  'content': '夜色笼罩大地，星星点点闪烁，寂静的夜晚，只有风声和虫鸣。'},
 'poem3': {'title': '春风',
  'author': '王五',
  'first_line': '春风拂过枝头',
  'content': '春风拂过枝头，吹落了树叶，带来了新的生机，让大地充满了活力。'}}

### 4.3 fallbacks方法

那这个时候可能就会思考，有没有什么方法，在不用改变太多代码的情况下，让早期的模型也能达到格式化输出的效果，而不是写复杂的格式化输出的代码去对结果进行操作。这时候我们就可以使用`fallbacks`的方式赋予早期模型这样格式化的能力，从结果我们也可以看出，我们成功使用`fallbacks`赋予了简单模型格式化的能力。

In [30]:
# 使用with_fallbacks机制
final_chain = simple_chain.with_fallbacks([chain])

# 调用final_chain的invoke方法，并传递challenge参数
final_chain.invoke(challenge)

JSONDecodeError: Extra data: line 15 column 1 (char 147)

### 4.4 fallbacks 是如何实现的？

当我们调用 LLM 时，经常会出现由于底层 API 问题、速率问题或者网络问题等原因，导致不能成功运行 LLM 。在这种情况下，我们就可以使用回退这种方法来解决这个问题，具体来说，他是通过使用另一种 LLM 来代替原先的不可运行的 LLM 产生结果，请看下面例子：

In [32]:
# from langchain_core.chat_models.openai import ChatOpenAI
from langchain_openai import ChatOpenAI
# from langchain_core.chat_models.anthropic import ChatAnthropic
from langchain_community.chat_models import ChatAnthropic
llm = ChatAnthropic()
model = llm.with_fallbacks([ChatOpenAI()])
model.invoke('hello')

ValidationError: 1 validation error for ChatAnthropic
__root__
  Did not find anthropic_api_key, please add an environment variable `ANTHROPIC_API_KEY` which contains it, or pass `anthropic_api_key` as a named parameter. (type=value_error)

在这种情况下，通常会优先使用 ChatAnthropic 进行回答，但是如果调用 ChatAnthropic 失败了，会回退到使用 ChatOpenAI 模型来生成响应。如果两种 LLM 都失败了，将会回退到一种硬编码响应。硬编码的默认响应用于处理异常情况或者在无法从外部资源获取所需信息时提供一个备用选项，例如 "Looks like our LLM providers are down. Here's a nice 🦜️ emoji for you instead."（看起来我们的 LLM 提供商出了问题，那么，这里有一个可爱的 🦜️ 表情符号给你。）

如果你想了解更多关于 fallbacks 的内容，请参考[官方文档](https://python.langchain.com/docs/guides/fallbacks)

## 五、接口 Interface

在使用LangChain中，存在许多接口，其中公开的标准接口包括：

- stream：流式返回输出内容
- invoke：输入调用chain
- batch：在输入列表中并行调用chain

这些也有相应的异步方法：

- astream：异步流式返回输出内容
- ainvoke：在输入上异步调用chain
- abatch：在输入列表中并行异步调用chain

首先我们定义给一个简单提示模板，也就是"给我讲一个关于{主题}的短笑话"，然后定义了一个简单的链`Chain = prompt | LLM | OutputParser`。

In [33]:
# 创建一个ChatPromptTemplate对象，使用模板"给我讲一个关于{topic}的短笑话"
prompt = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的短笑话"
)

# 创建一个ChatOpenAI模型
model = ChatOpenAI()

# 创建一个StrOutputParser对象
output_parser = StrOutputParser()

# 创建一个chain，将prompt、model和output_parser连接起来
chain = prompt | model | output_parser

### 5.1 invoke接口

接下来我们分别使用对应的接口，比如我们首先使用常规的`invoke`的调用，这个也是前面展现的方法，我们得到了对应结果。

In [34]:
chain.invoke({"topic": "熊"})

'为什么熊不喜欢在冬天洗澡？因为它们怕变成冰熊！😄😄😄'

### 5.2 batch接口

我们再尝试使用`batch`的接口，我们会发现大模型可以返回两个问题的答案，我们会给chain一个输入的列表，列表中可以包含多个问题，最后返回多个问题的答案。

In [35]:
chain.batch([{"topic": "熊"}, {"topic": "狐狸"}])

['为什么熊不喜欢在雨天出去玩耍？\n因为它怕变成“湿熊”！哈哈哈！', '为什么狐狸喜欢偷鸡呢？\n\n因为它们觉得这样能够变成“大神仙”！😄😄😄']

### 5.3 stream接口

接下来我们在看看`stream`接口，也就是流式输出内容，这样的功能很有必要，有时候可以免去用户等待的烦恼，让用户看到一个一个词蹦出来而不是一个空的屏幕，这样会带来更好的用户体验。

In [36]:
for t in chain.stream({"topic": "熊"}):
    print(t)


为
什
么
熊
不
喜
欢
雨
天
？
因
为
它
会
变
成
“
湿
熊
”
！
哈
哈
哈
！



### 5.4 异步接口

我们还可以尝试异步来调用，使用`ainvoke`来调用。


In [37]:
response = await chain.ainvoke({"topic": "熊"})
response

'为什么熊不喜欢在雨天出去？\n因为它怕会被“熊”湿！😄🐻'

## 六、英文提示词

**一、构建简单链**

In [38]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})

'Why do bears have hairy coats?\n\nFur protection!'

**2.1 构建简单文档数据库**

In [39]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [40]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [41]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [42]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

**3.2 使用RunnableMap**

In [43]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

**3.1 单函数绑定**

In [44]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [45]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

runnable = prompt | model

runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2a35b576-bbe0-4fe9-9670-f74b0a8aafc6-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

**3.2 多个函数绑定**

In [46]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [47]:
model = model.bind(functions=functions)

runnable = prompt | model

runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0dd47368-f747-471d-a149-22b415ba2c45-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117})

**4.1 使用早期模型格式化输出**

In [48]:
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

**早期模型不支持，会出现解码错误**

In [49]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

**4.2 较新的模型能够格式化输出**

In [50]:
model = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
chain = model | StrOutputParser() | json.loads

chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

**4.3 fallback机制**

In [51]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

**五、接口**

In [52]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

**5.1 invoke接口**

In [53]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food? Because they can't catch it!"

**5.2 batch接口**

In [54]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

**5.3 stream接口**

In [56]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 bring
 a
 flashlight
 to
 the
 party
?
 


Because
 he
 heard
 it
 was
 going
 to
 be
 a
 "
be
ary
"
 good
 time
!



**5.4 异步接口**

In [55]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear bring a backpack to the picnic?\nIn case he wanted to bear some snacks!'